# Aprendizaje Supervisado

Proyecto de gestión de inventarios usando SQL Server

## 1 Conexión

La conexión mostrada en este proyecto Actualmente se encuentra implementada para un servicio local, por lo tanto puede ser objeto de cambios en cuanto se despliegue la base de datos en un servidor EC2

In [ ]:
import pyodbc
import pandas as pd
import numpy as np

driver = 'SQL Server'
server = 'MARCO'
db1 = 'AI_Eafit'
tcon = 'yes'
uname = 'aieafit'
pword = 'marANT1502'

cnxn = pyodbc.connect(driver='{SQL Server}', host=server, database=db1,
                      trusted_connection=tcon, user=uname, password=pword)

# 2 Modelos

## 2.1 Regresión Logística

Una vez se ejecuta la conexión se deben seleccionar los datos establecidos en las variables de respuesta escogidas, este paso se dividirá por Queries de cada selección y se utilizará la biblioteca Microsoftml para implementar los modelos

### dbo.Fact_compra

In [ ]:
query_compras = "SELECT Id_Producto, Id_Bodega, Id_Comprador, Fecha_Orden_Compra, Compra_Cantidad, Compra_Pesos, Precio_Unitario_Moneda, Linea_Completa, Linea_Perfecta, Lead_Time_Teorico_Compra, Lead_Time_Real_Max_Compra FROM dbo.T_Fact_compra"
df_compras = pd.read_sql(query_compras, cnxn)

df_compras['Fecha_Orden_Compra'] = df_compras['Fecha_Orden_Compra'].str.replace(r'-', '')
df_compras['Fecha_Orden_Compra'] = df_compras['Fecha_Orden_Compra'].astype('int64')

df_compras.head()

#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [ ]:
from sklearn.model_selection import train_test_split
bc_train_compras, bc_test_compras = train_test_split(df_compras, test_size=0.2)
print("# of rows in training set = ",bc_train_compras.size)
print("# of rows in test set = ",bc_test_compras.size)

### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [ ]:
from microsoftml import rx_logistic_regression
from microsoftml import rx_ensemble
from microsoftml import rx_neural_network
from microsoftml import rx_fast_forest

features_compras = df_compras.columns.drop(["Linea_Perfecta"])
model_compras = rx_ensemble("Linea_Perfecta ~ " + "+".join(features_compras), data=bc_train_compras, trainers = [
                            rx_logistic_regression("Linea_Perfecta ~ " + "+".join(features_compras), data=bc_train_compras),
                            rx_neural_network("Linea_Perfecta ~ " + "+".join(features_compras), data=bc_train_compras),
                            rx_fast_forest("Linea_Perfecta ~ " + "+".join(features_compras), data=bc_train_compras)]
                           )

### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [ ]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()
print(model_compras.coef_)

### dbo.Fact_Ventas

In [ ]:
query_ventas = "SELECT Cod_Compañia, Id_Producto, Id_Vendedor, Id_Bodega, Fecha_Factura, Venta_Cantidad, Venta_Neta, Venta_Costo_Pesos, Venta_margen_pesos, Pedido_Numero_Linea FROM dbo.T_Fact_venta"
df_ventas = pd.read_sql(query_ventas, cnxn)

df_ventas['Fecha_Factura'] = df_ventas['Fecha_Factura'].str.replace(r'-', '')
df_ventas['Fecha_Factura'] = df_ventas['Fecha_Factura'].astype('int64')

df_ventas.head()

#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [ ]:
from sklearn.model_selection import train_test_split
bc_train_ventas, bc_test_ventas = train_test_split(df_ventas, test_size=0.2)
print("# of rows in training set = ",bc_train_ventas.size)
print("# of rows in test set = ",bc_test_ventas.size)

### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [ ]:
from microsoftml import rx_logistic_regression
from microsoftml import rx_ensemble
from microsoftml import rx_neural_network
from microsoftml import rx_fast_forest

features_ventas = df_ventas.columns.drop(["Id_Producto"])
model_ventas = rx_ensemble("Id_Producto~ " + "+".join(features_compras), method = 'multiClass', data=bc_train_ventas, trainers = [
                            rx_logistic_regression("Id_Producto ~ " + "+".join(features_ventas), data=bc_train_ventas, method = 'multiClass'),
                            rx_neural_network("Id_Producto ~ " + "+".join(features_ventas), data=bc_train_ventas, method = 'multiClass'),
                            rx_fast_forest("Id_Producto ~ " + "+".join(features_ventas), data=bc_train_ventas, method = 'multiClass')]
                           )

### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [ ]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()

### dbo.Fact_pedido

In [ ]:
query_pedido = "SELECT Id_Producto, Id_Vendedor, Id_Bodega, Fecha_Entrega_Planeada, Pedido_Descuento_Pesos, Pedido_Cantidad_Pendiente, conteo_linea_pedido, Linea_Completa, Linea_Perfecta, Pedido_Cantidad_Facturada, Pedido_Pesos_Facturados, Lead_time_Pedido_teorico, Lead_time_pedido_real_max FROM dbo.T_Fact_pedido"
df_pedido = pd.read_sql(query_pedido, cnxn)

df_pedido['Fecha_Entrega_Planeada'] = df_pedido['Fecha_Entrega_Planeada'].str.replace(r'-', '')
df_pedido['Fecha_Entrega_Planeada'] = df_pedido['Fecha_Entrega_Planeada'].astype('int64')

df_pedido.head()

#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [ ]:
from sklearn.model_selection import train_test_split
bc_train_pedido, bc_test_pedido = train_test_split(df_pedido, test_size=0.2)
print("# of rows in training set = ",bc_train_pedido.size)
print("# of rows in test set = ",bc_test_pedido.size)

### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [ ]:
from microsoftml import rx_logistic_regression
from microsoftml import rx_ensemble
from microsoftml import rx_neural_network
from microsoftml import rx_fast_forest

features_pedido = df_pedido.columns.drop(["Linea_Perfecta"])
model_pedido = rx_ensemble("Linea_Perfecta ~ " + "+".join(features_pedido), data=bc_train_pedido, trainers = [
                            rx_logistic_regression("Linea_Perfecta ~ " + "+".join(features_pedido), data=bc_train_pedido),
                            rx_neural_network("Linea_Perfecta ~ " + "+".join(features_pedido), data=bc_train_pedido),
                            rx_fast_forest("Linea_Perfecta ~ " + "+".join(features_pedido), data=bc_train_pedido)]
                           )

### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [ ]:
from microsoftml import rx_predict

prediction_pedido = rx_predict(model_pedido, data=bc_test_pedido)

prediction_pedido.head()

### dbo.Fact_Inventario

In [ ]:
query_inventario = "SELECT Fecha_Inventario, Id_Bodega, Id_Producto, Inventario_Cantidad, Inventario_Pesos FROM dbo.T_Fact_Inventario"
df_inventario = pd.read_sql(query_inventario, cnxn)

df_inventario['Fecha_Inventario'] = df_inventario['Fecha_Inventario'].str.replace(r'-', '')
df_inventario['Fecha_Inventario'] = df_inventario['Fecha_Inventario'].astype('int64')

df_inventario.head()

#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [ ]:
from sklearn.model_selection import train_test_split
bc_train_inventario, bc_test_inventario = train_test_split(df_inventario, test_size=0.2)
print("# of rows in training set = ",bc_train_inventario.size)
print("# of rows in test set = ",bc_test_inventario.size)

### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [ ]:
from microsoftml import rx_logistic_regression
from microsoftml import rx_ensemble
from microsoftml import rx_neural_network
from microsoftml import rx_fast_forest

features_inventario = df_inventarios.columns.drop(["Id_Producto"])
model_inventario = rx_ensemble("Id_Producto~ " + "+".join(features_inventario), method = 'multiClass', data=bc_train_inventario, trainers = [
                            rx_logistic_regression("Id_Producto ~ " + "+".join(features_inventario), data=bc_train_inventario, method = 'multiClass'),
                            rx_neural_network("Id_Producto ~ " + "+".join(features_inventario), data=bc_train_inventario, method = 'multiClass'),
                            rx_fast_forest("Id_Producto ~ " + "+".join(features_inventario), data=bc_train_inventario, method = 'multiClass')]
                           )

### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [ ]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()